In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path='/content/drive/My Drive/img'
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jul 24 14:12:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
imagePaths=[os.path.join(path,f) for f in os.listdir(path)]
training_data=[]
def create_training_data(ImageP):
  for imagepath in imagePaths:
    img_array=cv2.imread(imagepath)
    new_array=cv2.resize(img_array,(224,224))
    class_label=int(imagepath[-5])
    training_data.append([new_array,class_label])
create_training_data(imagePaths)

In [ ]:
import random
random.shuffle(training_data)
X=[]
y=[]
for image ,label in training_data:
  X.append(image)
  y.append(label)
X=np.array(X).reshape(-1,224,224,3).astype('float32')
y=np.array(y)  

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, y):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train=y[train_index]
    y_test=y[test_index]

In [ ]:
X_train=np.array(X_train).reshape(-1,224,224,3).astype('float32')
y_train=np.array(y_train)
X_test=np.array(X_test).reshape(-1,224,224,3).astype('float32')
y_test=np.array(y_test)

In [ ]:
X_train=keras.applications.vgg16.preprocess_input(X_train)
X_test=keras.applications.vgg16.preprocess_input(X_test)

In [ ]:
vgg=keras.applications.vgg16.VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
my_smaller_model =keras.models.Model(vgg.input,vgg.get_layer("block3_conv3").output)
my_smaller_model.trainable = False

In [ ]:
for layer in my_smaller_model.layers:
     layer.trainable = False

In [ ]:
base_model_output = my_smaller_model.output
gap = keras.layers.GlobalAveragePooling2D()(base_model_output)
fc = keras.layers.Dense(units=256, activation="relu")(gap)
output = keras.layers.Dense(1, activation="sigmoid")(fc)

In [ ]:
transfer_model = keras.models.Model(my_smaller_model.input, output)

In [ ]:
transfer_model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import StratifiedKFold
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
restore_best_weights=True)
skfolds = StratifiedKFold(n_splits=5)
for train_index, test_index in skfolds.split(X_train, y_train):
  X_train_folds = X_train[train_index]
  y_train_folds = y_train[train_index]
  X_test_fold = X_train[test_index]
  y_test_fold = y_train[test_index]
  transfer_model.fit(X_train,y_train,batch_size=1,epochs=30,validation_split=0.1,callbacks=[early_stopping_cb])
  y_pred = transfer_model.predict(X_test_fold)
  n_correct = sum(y_pred == y_test_fold)

Epoch 1/30
187/187 [==============================] - 12s 7ms/step - loss: 30.2201 - accuracy: 0.5668 - val_loss: 2.2045 - val_accuracy: 0.8095
Epoch 2/30
187/187 [==============================] - 1s 6ms/step - loss: 6.7243 - accuracy: 0.7273 - val_loss: 0.4671 - val_accuracy: 0.9524
Epoch 3/30
187/187 [==============================] - 1s 6ms/step - loss: 6.7431 - accuracy: 0.8021 - val_loss: 0.0615 - val_accuracy: 0.9524
Epoch 4/30
187/187 [==============================] - 1s 5ms/step - loss: 8.4032 - accuracy: 0.7540 - val_loss: 14.9733 - val_accuracy: 0.4762
Epoch 5/30
187/187 [==============================] - 1s 5ms/step - loss: 8.7544 - accuracy: 0.7166 - val_loss: 0.7360 - val_accuracy: 0.9524
Epoch 6/30
187/187 [==============================] - 1s 5ms/step - loss: 1.6488 - accuracy: 0.8770 - val_loss: 8.5883 - val_accuracy: 0.6190
Epoch 7/30
187/187 [==============================] - 1s 5ms/step - loss: 3.3653 - accuracy: 0.8235 - val_loss: 0.0063 - val_accuracy: 1.0000
Epo

In [ ]:
my_smaller_model.trainable = True
for layer in my_smaller_model.layers:
     layer.trainable = True 

In [ ]:
ADAM= keras.optimizers.Adam(learning_rate=0.00001)
transfer_model.compile(loss="binary_crossentropy",optimizer=ADAM,metrics=['accuracy'])

In [ ]:
transfer_model.fit(X_train,y_train,batch_size=10,epochs=20,validation_split=0.1,callbacks=[early_stopping_cb])

Epoch 1/20
19/19 [==============================] - 5s 108ms/step - loss: 0.2210 - accuracy: 0.9465 - val_loss: 0.2013 - val_accuracy: 0.9048
Epoch 2/20
19/19 [==============================] - 1s 66ms/step - loss: 0.1204 - accuracy: 0.9679 - val_loss: 0.1291 - val_accuracy: 0.9524
Epoch 3/20
19/19 [==============================] - 1s 66ms/step - loss: 0.1378 - accuracy: 0.9679 - val_loss: 0.1666 - val_accuracy: 0.9048
Epoch 4/20
19/19 [==============================] - 1s 66ms/step - loss: 0.1096 - accuracy: 0.9679 - val_loss: 0.1113 - val_accuracy: 1.0000
Epoch 5/20
19/19 [==============================] - 1s 65ms/step - loss: 0.1588 - accuracy: 0.9091 - val_loss: 0.4287 - val_accuracy: 0.8095
Epoch 6/20
19/19 [==============================] - 1s 66ms/step - loss: 0.1445 - accuracy: 0.9412 - val_loss: 0.0906 - val_accuracy: 0.9524
Epoch 7/20
19/19 [==============================] - 1s 67ms/step - loss: 0.0588 - accuracy: 0.9786 - val_loss: 0.0684 - val_accuracy: 0.9524
Epoch 8/20
1

In [ ]:
mse_test = transfer_model.evaluate(X_test, y_test)

2/2 [==============================] - 1s 432ms/step - loss: 0.0890 - accuracy: 0.9808
